## Make GSM Names Template
We made a file that has the names and numbers masked out. This code will go through and fill in the names with the orignal names from unedited_100.jsonl, allowing us to run GSM Numbers independent of GSM Names.

Note the make_mapping makes the assumption that there is not going to be two variabilized portions back to back without a space. So if there is something like "{person1}{person2}", it would create an error. I found that this was never the case, so I ignored that edge case as it greatly simplified the code to do so.

In [36]:
import re

def make_mapping(var_str, filled_str):
    mapping = {}
    parts = re.split(r'(\{[^}]+\})', var_str)
    filled_str_i = 0
    for part_i in range(len(parts)):
        # make mapping
        if parts[part_i].startswith('{'):
            # find next section in filled_str
            next_part = parts[part_i+1]
            next_part_i = filled_str.find(next_part, filled_str_i)
            if parts[part_i] not in mapping:
                mapping[parts[part_i]] = filled_str[filled_str_i:next_part_i]
            filled_str_i = next_part_i
        else:
            # skip over this section
            filled_str_i += len(parts[part_i])
    return mapping

In [35]:
import pandas as pd

need_filling_data = pd.read_json('templates/name_number_templates.json')
name_templates = pd.read_json('templates/name_templates.jsonl', lines=True)
orig_data = pd.read_json('datasets/unedited_100.jsonl', lines=True)

# Get the first 20 values from the DataFrame
json_data = need_filling_data.head(20)

for i in range(len(json_data)):
    var_str = name_templates.iloc[i]['question']
    filled_str = orig_data.iloc[i]['question']
    mapping = make_mapping(var_str, filled_str)
    for key in mapping:
        need_filling_data.iloc[i]['question'] = need_filling_data.iloc[i]['question'].replace(key, mapping[key])
        need_filling_data.iloc[i]['answer'] = need_filling_data.iloc[i]['answer'].replace(key, mapping[key])
need_filling_data.to_json('templates/number_templates.jsonl', orient='records', lines=True)